# Running ASFMAUS

In [1]:
using StochasticDiffEq
using DiffEqCallbacks
using DifferentialEquations
using Plots
using LinearAlgebra
using DelimitedFiles
using DataFrames
using LinearAlgebra
using SparseArrays
using Distributions

In [7]:
directory_path = "/home/callum/ASF/"

"/home/callum/ASF/"

In [8]:
include("$(directory_path)Modules/ASF_input.jl");
include("$(directory_path)Modules/ASF_model.jl");
include("$(directory_path)Modules/ASF_analysis.jl");

In [9]:
input_1 = ASF_Inputs.Model_Data(directory_path); #creating parameters to run model

In [6]:
nt = input_1.Parameters.Populations.cum_sum[end] #total number of groups and farms
nc = 5 #number of classes
eqs = 11 #number of processes
n_ens = 100 #number of runs in ensemble

dc = sparse(zeros(nt*nc,nt*eqs))

#const _dc = zeros(nt*5,nt*10)

dc[0*nc*nt+1:nc*nt*eqs+nc:end] .= 1
dc[1*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[2*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+1:nc*nt*eqs+nc:end] .= 1

dc[2*nc*nt+2:nc*nt*eqs+nc:end] .= 1
dc[3*nc*nt+2:nc*nt*eqs+nc:end] .= -1
dc[4*nc*nt+2:nc*nt*eqs+nc:end] .= -1

dc[4*nc*nt+3:nc*nt*eqs+nc:end] .= 1
dc[5*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[6*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[7*nc*nt+3:nc*nt*eqs+nc:end] .= -1

dc[7*nc*nt+4:nc*nt*eqs+nc:end] .= 1
dc[8*nc*nt+4:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+4:nc*nt*eqs+nc:end] .= -1


dc[5*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[6*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[9*nc*nt+5:nc*nt*eqs+nc:end] .= -1;

In [5]:
dc

10×22 SparseMatrixCSC{Float64, Int64} with 34 stored entries:
⠉⠓⠦⢤⡀⡁⠀⠀⠀⠀⠀
⠀⠀⠈⠁⠈⠐⠲⢤⣀⡀⠐
⠀⠀⠀⠀⠀⠀⠀⠀⠒⠉⠊

In [6]:
function regular_c(du,u,p,t,counts,mark)  
    mul!(du,dc,counts)
end

function prob_func(prob,i,repeat) #function to change parameters while keeping network the same
                ASF_Model.reparam!(input_1)
                remake(prob,p=input_1.Parameters)
end 

prob_func (generic function with 1 method)

## Running the Model

### Frequency run

In [9]:
#single run with frequency inter group interections
rj = RegularJump(ASF_Model.frequency_rate,regular_c,eqs*nt)

jumps = JumpSet(rj)

prob = DiscreteProblem(input_1.U0,input_1.Time,input_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)

#sol = solve(jump_prob,SimpleTauLeaping(), dt=.1)

ensemble_prob_m = EnsembleProblem(jump_prob)#,prob_func=prob_func)
output_f = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= 20,dt=1);

In [10]:
ASF_Analysis.analyse_out(output_f, input_1.Parameters.Populations)

------------------------------------------------
Run Stats:
1 Populations
20 Runs
------------------------------------------------
% of groups exposed to ASF in each population:
[99.45]
------------------------------------------------
% of groups that survive in each population:
[36.95]
------------------------------------------------


### Density run

In [7]:

rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)

jumps = JumpSet(rj)
#ensemble run with density inter group interections

prob = DiscreteProblem(input_1.U0,input_1.Time,input_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj) #rng=rng)
#sol = solve(jump_prob,SimpleTauLeaping(), dt=.1)

ensemble_prob_m = EnsembleProblem(jump_prob,prob_func=prob_func)
output = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= 100,EnsembleThreads(),dt=1);

In [8]:
ASF_Analysis.analyse_out(output, input_1.Parameters.Populations)

------------------------------------------------
Run Stats:
1 Populations
100 Runs
------------------------------------------------
% of groups exposed to ASF in each population:
[95.68]
------------------------------------------------
% of groups that survive in each population:
[61.17]
------------------------------------------------


## Running Multiple Networks

In [10]:
function sensitivity(n_networks, n_dist, n_runs, path, name)
    
    dir = "$(path)Results/$(name)"

    isdir(dir) || mkdir(dir)
    cd(dir)

    for i = 1:n_networks
        inputs = ASF_Inputs.Model_Data(directory_path); #creating parameters to run model

        for j = 1:n_dist

            rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)
            jumps = JumpSet(rj)
            prob = DiscreteProblem(inputs.U0,inputs.Time,inputs.Parameters)
            jump_prob = JumpProblem(prob,Direct(),rj) 
            ensemble_prob_m = EnsembleProblem(jump_prob)
            outputs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1);

            summary = ASF_Analysis.quick_analysis(outputs) #update to new function from module (ASF_Model.quick_analysis(outputs))

            writedlm( "Results_$(i)_$(j).csv",  summary, ',')
            ASF_Model.reparam!(inputs) #re-doing params with
        end
        println("$(100*i/n_networks) %")

    end
    
end

sensitivity (generic function with 1 method)

In [11]:
sensitivity(10,10,100,directory_path, "Death/death_hundred") # running 

10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 %


In [92]:
function sensitivity2(n_networks, n_dist, n_runs, path, name)
    
    dir = "$(path)Results/$(name)"

    isdir(dir) || mkdir(dir)
    cd(dir)

    for i = 1:n_networks

        inputs = ASF_Inputs.Model_Data(directory_path); #creating parameters to run model

        for j = 1:n_dist

            rj = RegularJump(ASF_Model.frequency_rate,regular_c,eqs*nt)
            jumps = JumpSet(rj)
            prob = DiscreteProblem(inputs.U0,inputs.Time,inputs.Parameters)
            jump_prob = JumpProblem(prob,Direct(),rj) 
            ensemble_prob_m = EnsembleProblem(jump_prob)
            outputs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1);

            summary = ASF_Analysis.quick_analysis(outputs) #update to new function from module (ASF_Model.quick_analysis(outputs))

            writedlm( "Results_$(i)_$(j).csv",  summary, ',')
            ASF_Model.reparam!(inputs) #re-doing params with
        end
    end
    
end

sensitivity2 (generic function with 1 method)

In [94]:
sensitivity2(20,20,100,directory_path, "Frequency") # running 

Need to Reduce Timestep
Need to Reduce Timestep
Need to Reduce Timestep


## Analysing Output

In [18]:
density = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Baseline//");
connect_5 = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Connect_5//");


In [19]:
density_r = ASF_Analysis.three_statistics(density,20, p_out = true);

Exposed!
Mean- 91.88 & 91.75 - 92.01

Alive!
Mean- 41.8 & 41.72 - 41.88

Time!
Mean- 262.43 & 261.91 - 262.94

Filtered

Exposed!
Mean- 93.69 & 93.67 - 93.72

Alive!
Mean- 40.93 & 40.88 - 40.98

Time!
Mean- 266.66 & 266.23 - 267.09


In [20]:
connect_5_r = ASF_Analysis.three_statistics(connect_5,20, p_out = true);

Exposed!
Mean- 92.93 & 92.79 - 93.07

Alive!
Mean- 41.1 & 40.98 - 41.21

Time!
Mean- 262.15 & 261.27 - 263.03

Filtered

Exposed!
Mean- 93.42 & 93.35 - 93.48

Alive!
Mean- 40.87 & 40.77 - 40.97

Time!
Mean- 263.28 & 262.45 - 264.11
